<a href="https://colab.research.google.com/github/keripikkaneboo/Hands-On-Machine-Learning-O-Reilly-/blob/main/18.%20Chapter18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Bab 18: Reinforcement Learning

Bab ini membahas **Reinforcement Learning (RL)**, sebuah cabang Machine Learning di mana sebuah *software agent* belajar untuk mengambil tindakan dalam sebuah *environment* untuk memaksimalkan *reward* kumulatif dari waktu ke waktu.

* **Konsep Inti RL**:
    * ***Agent***: Entitas yang belajar dan membuat keputusan (misalnya, program yang mengontrol karakter game).
    * ***Environment***: Dunia di mana agent beroperasi (misalnya, game Atari).
    * ***Action***: Keputusan yang diambil oleh agent.
    * ***Observation***: Informasi yang diterima agent dari environment.
    * ***Reward***: Umpan balik (positif atau negatif) yang diterima agent setelah mengambil tindakan.
    * ***Policy***: Strategi yang digunakan agent untuk memilih tindakan berdasarkan observasi. Tujuannya adalah untuk menemukan *policy* yang memaksimalkan total *reward*.

* **OpenAI Gym**: Sebuah *toolkit* yang menyediakan berbagai lingkungan simulasi (game, robotika, dll.) untuk mengembangkan dan membandingkan algoritma RL.

* **Tantangan Credit Assignment**: Kesulitan utama dalam RL adalah mengetahui tindakan mana yang menyebabkan sebuah *reward*. Seringkali, *reward* datang dengan penundaan. Solusinya adalah dengan mengevaluasi setiap tindakan berdasarkan jumlah *reward* masa depan yang didiskontokan (*discounted future rewards*).

* **Dua Pendekatan Utama**:
    1.  **Policy Gradients (PG)**:
        * Algoritma yang secara langsung mencoba mengoptimalkan parameter dari sebuah *policy*.
        * Jaringan saraf digunakan sebagai *policy network* yang memetakan observasi ke probabilitas tindakan.
        * Training dilakukan dengan cara: menjalankan beberapa episode, menghitung *advantage* (keuntungan) dari setiap tindakan, lalu memperbarui bobot jaringan untuk membuat tindakan yang "baik" (dengan *advantage* positif) lebih mungkin terjadi di masa depan.
    2.  **Q-Learning**:
        * Alih-alih belajar *policy* secara langsung, algoritma ini belajar untuk mengestimasi nilai optimal dari setiap pasangan *state-action*, yang disebut **Q-Value**.
        * Q-Value `Q(s, a)` merepresentasikan total *reward* masa depan yang diharapkan jika agent berada di *state* `s`, mengambil *action* `a`, dan kemudian bertindak secara optimal.
        * ***Deep Q-Network (DQN)***: Menggunakan jaringan saraf dalam untuk mengaproksimasi Q-Values, yang memungkinkan penerapan pada lingkungan dengan *state space* yang sangat besar (seperti game dari layar piksel mentah).
        * ***Replay Buffer (Experience Replay)***: Teknik krusial di mana pengalaman (transisi *state*, *action*, *reward*) disimpan dalam sebuah buffer. Model dilatih pada sampel acak dari buffer ini untuk mengurangi korelasi antar pengalaman dan menstabilkan training.

* **TF-Agents**: Library tingkat tinggi dari Google untuk RL yang menyederhanakan implementasi agen kompleks, lingkungan, dan *training loop*.

### 1. Pengenalan OpenAI Gym
Contoh dasar penggunaan lingkungan `CartPole`, di mana tujuannya adalah menyeimbangkan tiang di atas kereta.

```python
import gym
import numpy as np

# Membuat environment
env = gym.make("CartPole-v1")
obs = env.reset()

print("Initial observation:", obs)

# Menjalankan satu langkah dengan aksi acak (0=kiri, 1=kanan)
action = env.action_space.sample()
obs, reward, done, info = env.step(action)

print("New observation:", obs)
print("Reward:", reward)
print("Done:", done)

env.close()
```

### 2. Policy Gradients (PG)
Kita akan membangun *policy network* sederhana dan melatihnya dengan algoritma REINFORCE.

```python
import tensorflow as tf
from tensorflow import keras

# 1. Membuat Policy Network
n_inputs = env.observation_space.shape[0] # 4 untuk CartPole
n_outputs = env.action_space.n # 2 untuk CartPole

model_pg = keras.models.Sequential([
    keras.layers.Dense(5, activation="elu", input_shape=[n_inputs]),
    keras.layers.Dense(1, activation="sigmoid"), # Probabilitas aksi "kiri"
])

# 2. Fungsi untuk menjalankan satu episode dan mengumpulkan reward & gradien
def play_one_step(env, obs, model, loss_fn):
    with tf.GradientTape() as tape:
        left_proba = model(obs[np.newaxis])
        action = (tf.random.uniform([1, 1]) > left_proba)
        y_target = tf.constant([[1.]]) - tf.cast(action, tf.float32)
        loss = tf.reduce_mean(loss_fn(y_target, left_proba))
    
    grads = tape.gradient(loss, model.trainable_variables)
    obs, reward, done, info = env.step(int(action[0, 0].numpy()))
    return obs, reward, done, grads

# 3. Fungsi untuk menghitung discounted rewards
def discount_rewards(rewards, discount_factor):
    discounted = np.array(rewards)
    for step in range(len(rewards) - 2, -1, -1):
        discounted[step] += discounted[step + 1] * discount_factor
    return discounted

# 4. Fungsi untuk menormalisasi rewards (menjadi advantage)
def discount_and_normalize_rewards(all_rewards, discount_factor):
    all_discounted_rewards = [discount_rewards(rewards, discount_factor)
                              for rewards in all_rewards]
    flat_rewards = np.concatenate(all_discounted_rewards)
    reward_mean = flat_rewards.mean()
    reward_std = flat_rewards.std()
    return [(discounted - reward_mean) / reward_std
            for discounted in all_discounted_rewards]

# Kerangka Training Loop (kode penuh ada di notebook)
# optimizer = keras.optimizers.Adam(learning_rate=0.01)
# loss_fn = keras.losses.binary_crossentropy

# for iteration in range(n_iterations):
#     all_rewards, all_grads = play_multiple_episodes(...)
#     all_final_rewards = discount_and_normalize_rewards(...)
#     # Menghitung gradien rata-rata yang sudah dibobot oleh advantage
#     # Menerapkan gradien ke model
print("Kerangka untuk Policy Gradients telah disiapkan.")
```

### 3. Deep Q-Network (DQN)
Kita akan membuat DQN untuk masalah yang sama. Kuncinya adalah *replay buffer* dan pembaruan target berdasarkan *Bellman equation*.

```python
from collections import deque

# 1. Membuat Replay Buffer
replay_buffer = deque(maxlen=2000)

# 2. Membuat Deep Q-Network (DQN)
# Model ini akan memprediksi Q-Value untuk setiap aksi
model_dqn = keras.models.Sequential([
    keras.layers.Dense(32, activation="elu", input_shape=[n_inputs]),
    keras.layers.Dense(32, activation="elu"),
    keras.layers.Dense(n_outputs)
])

# 3. Fungsi untuk memilih aksi (Epsilon-Greedy Policy)
def epsilon_greedy_policy(state, epsilon=0):
    if np.random.rand() < epsilon:
        return np.random.randint(n_outputs)
    else:
        Q_values = model_dqn.predict(state[np.newaxis])
        return np.argmax(Q_values[0])

# 4. Kerangka Training Step
# def training_step(batch_size):
#     # 1. Sampel pengalaman dari replay buffer
#     experiences = sample_experiences(batch_size)
#     # 2. Hitung target Q-Values menggunakan Bellman equation
#     #    target_Q_values = rewards + discount_factor * np.max(next_Q_values, axis=1)
#     # 3. Latih model DQN dengan Gradient Descent pada (target_Q_values - predicted_Q_values)
print("\nKerangka untuk Deep Q-Network (DQN) telah disiapkan.")
```
Training RL seringkali tidak stabil dan sangat sensitif terhadap *hyperparameter*. Menggunakan library yang sudah matang seperti **TF-Agents** sangat direkomendasikan untuk proyek yang lebih serius karena ia mengimplementasikan banyak perbaikan dan stabilisasi (seperti *Double DQN*, *Dueling DQN*, dll).
